In [1]:
import subprocess
import re
import os
import urllib
from bs4 import BeautifulSoup as bs

In [2]:
MAX_TRAILES = 10

In [3]:
def get_html(url):
    try:
        return subprocess.check_output(f'curl -s {url}', shell=True), 0
    except subprocess.CalledProcessError as grepexc:                                                                                                   
        return None, grepexc.returncode
    
def fetch_q(q, max_trailes=10):
    dict_q = {'q': q, 'status': None, 'returncode':None, 'results': []}
    for i in range(max_trailes):
        html_q, dict_q['returncode'] = get_html(f'https://html.duckduckgo.com/html/?q={urllib.parse.quote(q)}')
        if dict_q['returncode'] != 0:
            dict_q['status'] = 'Failed'
        else:
            bs_q = bs(html_q)
            results = bs_q.find_all('a', attrs={'class': 'result__snippet'})
            if results == []:
                dict_q['status'] = 'Failed'
            else:
                dict_q['status'] = 'Success'
                for i, result in enumerate(results):
                    dict_q['results'].append({})
                    dict_q['results'][i]['idx'] = i
                    dict_q['results'][i]['link'] = urllib.parse.unquote(re.search(r'uddg.*&rut', result['href'])[0][5:-4])
                    dict_q['results'][i]['snippet'] = result.get_text()
                    
        if dict_q['status'] == 'Success':
            break

    return dict_q

In [4]:
queries = ['ab', 'kfc', 'mcdonalds', 'Pizza hot', 'xyz', 'word']
[fetch_q(q, max_trailes=MAX_TRAILES) for q in queries]

# output structure
'''
[
    {
        'q': q, # Query
        'status': None, # Sucess/Faield
        'returncode': None, # returncode from subprocess
        'results': [
            {
                'idx': i, # results counter
                'link': url, # result url
                'snippet': snippet, # result snippet
            },
            .
            .
            .
        ]
    },
    .
    .
    .
]
'''

[{'q': 'ab',
  'status': 'Success',
  'returncode': 0,
  'results': [{'idx': 0,
    'link': 'https://en.wikipedia.org/wiki/AB',
    'snippet': 'AB, Ab, or ab may refer to: Abhishek Bachchan, an Indian actor. Alter Bridge, an American hard rock band. American Bandstand, a music-performance television show. Amitabh Bachchan, an Indian actor. Analecta Bollandiana, an academic journal.'},
   {'idx': 1,
    'link': 'https://en.wiktionary.org/wiki/ab',
    'snippet': 'Abbreviation of abdominal muscles. ab (plural abs). (informal) abdominal muscle. [Mid 20th century.] 2006, H. Peter Steeves, The Things Themselves, page 75: The bikinied models in most of the ESPN2 shows have abs. Many of the malnourished bikinied models in the commercials have visible rib cages.'},
   {'idx': 2,
    'link': 'https://www.youtube.com/watch?v=NMgWyTivrTI',
    'snippet': '22 Ab Exercises Ranked (Worst to Best!) Смотреть позже.'},
   {'idx': 3,
    'link': 'https://clinic-a-plus.ru/articles/gematologiya/2500-grupp